In [0]:
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60104384.dfs.core.windows.net",
    "pZcTKMsN83zedQA3LcYU2xsID/DzMx//bY0Is0yuMyWoXNoHgx1Rb17QRSzW9ksEoZu8KqCQPtOU+ASto/2ySQ=="
)

In [0]:
books = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/books/"
)

authors = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/authors/"
)

books.show(5)
authors.show(5)
books.printSchema()
authors.printSchema()

+----------+------------------+---------+------------+-------------+-----+-------+----------+--------+--------------+-----------+----------------+--------------------+---------+--------------------+---------+----+--------------------+---------+---------------+-------------+-----------------+-------------------+----------------+--------------------+--------------------+-------+-------------+-------+--------------------+--------------------+
|      isbn|text_reviews_count|series'][|country_code|language_code|count|   name|      asin|is_ebook|average_rating|kindle_asin|similar_books'][|         description|   format|                link|author_id|role|           publisher|num_pages|publication_day|       isbn13|publication_month|edition_information|publication_year|                 url|           image_url|book_id|ratings_count|work_id|               title|title_without_series|
+----------+------------------+---------+------------+-------------+-----+-------+----------+--------+----------

In [0]:
#to verify 
dbutils.fs.ls("abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/")

[FileInfo(path='abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/authors/', name='authors/', size=0, modificationTime=1762257752000),
 FileInfo(path='abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/authors_delta/', name='authors_delta/', size=0, modificationTime=1762435905000),
 FileInfo(path='abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/books/', name='books/', size=0, modificationTime=1762257732000),
 FileInfo(path='abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/books_delta/', name='books_delta/', size=0, modificationTime=1762435186000),
 FileInfo(path='abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/', name='reviews/', size=0, modificationTime=1762257741000),
 FileInfo(path='abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews_delta/', name='reviews_delta/', size=0, modificationTime=1762435915000)]

In [0]:
fixed_reviews.write.mode("overwrite").parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/"
)



In [0]:
reviews = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/"
)
reviews.show(5)


+--------------------+--------+--------------------+------+--------------------+--------------------+-------+
|           review_id| book_id|             user_id|rating|         review_text|          date_added|n_votes|
+--------------------+--------+--------------------+------+--------------------+--------------------+-------+
|c699efe94730c5e7b...|28670421|16c6b7c60483be209...|     4|Great continuatio...|Wed Aug 10 18:36:...|      1|
|22d969d60581e7161...|29227384|16c6b7c60483be209...|     4|I loved reading t...|Tue Aug 02 16:01:...|      0|
|e1a2ab081147a2a4e...|30973139|16c6b7c60483be209...|     5|Oh my heart, I ju...|Tue Aug 02 14:23:...|      3|
|bfa34c7cb9d3dc23a...|11178493|16c6b7c60483be209...|     4|My first Marcia L...|Sun Jul 31 20:46:...|      0|
|e4adf5d2bc2c76f1a...|  344474|16c6b7c60483be209...|     4|I picked this boo...|Sat Jul 30 19:13:...|      1|
+--------------------+--------+--------------------+------+--------------------+--------------------+-------+
only showi

In [0]:
reviews = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/"
)
reviews.show(5)


+--------------------+--------+--------------------+------+--------------------+--------------------+-------+
|           review_id| book_id|             user_id|rating|         review_text|          date_added|n_votes|
+--------------------+--------+--------------------+------+--------------------+--------------------+-------+
|c699efe94730c5e7b...|28670421|16c6b7c60483be209...|     4|Great continuatio...|Wed Aug 10 18:36:...|      1|
|22d969d60581e7161...|29227384|16c6b7c60483be209...|     4|I loved reading t...|Tue Aug 02 16:01:...|      0|
|e1a2ab081147a2a4e...|30973139|16c6b7c60483be209...|     5|Oh my heart, I ju...|Tue Aug 02 14:23:...|      3|
|bfa34c7cb9d3dc23a...|11178493|16c6b7c60483be209...|     4|My first Marcia L...|Sun Jul 31 20:46:...|      0|
|e4adf5d2bc2c76f1a...|  344474|16c6b7c60483be209...|     4|I picked this boo...|Sat Jul 30 19:13:...|      1|
+--------------------+--------+--------------------+------+--------------------+--------------------+-------+
only showi

In [0]:
#HomeWork1 update to fix the time added

from pyspark.sql.functions import (
    col,
    substring,
    regexp_extract,
    when,
    concat,
    lit,
    to_date
)


# STEP 0: Load silver books & authors (from ADF)

books = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/books/"
)

authors = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/authors/"
)

# Rename authors.name -> author_name to avoid ambiguity after joins
authors = authors.select(
    col("author_id"),
    col("name").alias("author_name")
)

print("Books schema:")
books.printSchema()
print("Authors schema (renamed):")
authors.printSchema()


# STEP 1: Build fixed reviews from RAW JSON (includes date_added, n_votes)

raw_reviews = spark.read.json(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/raw/reviews/goodreads_reviews_dedup.json"
)

fixed_reviews = raw_reviews.select(
    col("review_id").cast("string"),
    col("book_id").cast("string"),
    col("user_id").cast("string"),
    col("rating").cast("long").alias("rating"),
    col("review_text").cast("string"),
    col("date_added").cast("string"),
    col("n_votes").cast("long").alias("n_votes")
)

print("Fixed Reviews schema (from raw JSON):")
fixed_reviews.printSchema()
fixed_reviews.show(5)


# STEP 1b: Overwrite SILVER /processed/reviews/ as PARQUET

fixed_reviews.write.mode("overwrite").parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/"
)


# STEP 2: Reload fixed reviews from /processed/reviews/ (Parquet)

reviews = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/"
)

print("Reviews schema (fixed from /processed/reviews/):")
reviews.printSchema()
reviews.show(5)


# STEP 2b: Manually convert date_added string -> yyyy-MM-dd


reviews_with_date = (
    reviews
    .withColumn("month_abbr", substring(col("date_added"), 5, 3))
    .withColumn("day_str", substring(col("date_added"), 9, 2))
    .withColumn("year_str", regexp_extract(col("date_added"), "(\\d{4})$", 1))
)

# Map month abbreviation -> number
reviews_with_date = reviews_with_date.withColumn(
    "month_num",
    when(col("month_abbr") == "Jan", "01")
    .when(col("month_abbr") == "Feb", "02")
    .when(col("month_abbr") == "Mar", "03")
    .when(col("month_abbr") == "Apr", "04")
    .when(col("month_abbr") == "May", "05")
    .when(col("month_abbr") == "Jun", "06")
    .when(col("month_abbr") == "Jul", "07")
    .when(col("month_abbr") == "Aug", "08")
    .when(col("month_abbr") == "Sep", "09")
    .when(col("month_abbr") == "Oct", "10")
    .when(col("month_abbr") == "Nov", "11")
    .when(col("month_abbr") == "Dec", "12")
)

# Build yyyy-MM-dd string and convert to date
reviews_with_date = reviews_with_date.withColumn(
    "date_added_clean",
    to_date(
        concat(col("year_str"), lit("-"), col("month_num"), lit("-"), col("day_str")),
        "yyyy-MM-dd"
    )
)

print("Cleaned date preview:")
reviews_with_date.select("date_added", "date_added_clean").show(5, truncate=False)


# STEP 3: Build curated_reviews (GOLD layer)

curated_reviews = (
    reviews_with_date
    .join(books, on="book_id", how="inner")
    .join(authors, on="author_id", how="inner")
    .select(
        col("review_id"),
        col("book_id"),
        col("title"),
        col("author_id"),
        col("author_name"),
        col("user_id"),
        col("rating"),
        col("review_text"),
        col("language_code").alias("language"),
        col("n_votes"),
        col("date_added_clean").alias("date_added")  # final yyyy-MM-dd DATE
    )
)

print("Curated reviews schema:")
curated_reviews.printSchema()
curated_reviews.show(5)
print("Total rows in curated_reviews:", curated_reviews.count())


# STEP 4: Write GOLD as Delta (no external-location SQL)

gold_path = "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/gold/curated_reviews/"

curated_reviews.write.format("delta") \
    .option("overwriteSchema", "true") \
    .mode("overwrite") \
    .save(gold_path)

# Optional: register a TEMP VIEW so you can query it with SQL in this notebook
spark.read.format("delta").load(gold_path).createOrReplaceTempView("curated_reviews")





Books schema:
root
 |-- isbn: string (nullable = true)
 |-- text_reviews_count: string (nullable = true)
 |-- series'][: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- count: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- is_ebook: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- kindle_asin: string (nullable = true)
 |-- similar_books'][: string (nullable = true)
 |-- description: string (nullable = true)
 |-- format: string (nullable = true)
 |-- link: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- role: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- num_pages: string (nullable = true)
 |-- publication_day: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- publication_month: string (nullable = true)
 |-- edition_information: string (nullable = true)
 |-- publication_

In [0]:
 spark.sql("SELECT * FROM curated_reviews LIMIT 10").show()

+--------------------+--------+--------------------+---------+----------------+--------------------+------+--------------------+--------+-------+----------+
|           review_id| book_id|               title|author_id|     author_name|             user_id|rating|         review_text|language|n_votes|date_added|
+--------------------+--------+--------------------+---------+----------------+--------------------+------+--------------------+--------+-------+----------+
|073abf39809418df5...|11769081|The Shape of Desi...|    28544|    Sharon Shinn|d7879573928a367ed...|     4|Some how this fan...|     eng|      0|2012-03-31|
|3730de47e1474172a...|13257056|Unstoppable (Trac...|    84181|   Laura Griffin|d7879573928a367ed...|     4|A satisfying litt...|     eng|      0|2012-03-29|
|2d3a5caf076017d85...| 9816552|The Urban Fantasy...|  1067608| Peter S. Beagle|d7879573928a367ed...|     4|A very interestin...|   en-US|      1|2011-07-13|
|d7965caa5d2144af8...| 6767883|Shady Lady (Corin...|   835

In [0]:
from pyspark.sql.functions import col, length, trim, count, when

reviews = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/"
)

reviews.show(5, truncate=False)
reviews.printSchema()

total_rows = reviews.count()
null_review_id = reviews.filter(col("review_id").isNull()).count()
null_book_id = reviews.filter(col("book_id").isNull()).count()
null_user_id = reviews.filter(col("user_id").isNull()).count()
null_rating = reviews.filter(col("rating").isNull()).count()
empty_text = reviews.filter( (col("review_text").isNull()) | (trim(col("review_text")) == "") ).count()

print(f"Total rows: {total_rows}")
print(f"NULL review_id: {null_review_id}, NULL book_id: {null_book_id}, NULL user_id: {null_user_id}, NULL rating: {null_rating}")
print(f"Empty/NULL review_text: {empty_text}")

In [0]:
from pyspark.sql.functions import col, trim, length

df = reviews

# 1) Drop rows missing critical keys
df = df.filter(
    col("review_id").isNotNull() &
    col("book_id").isNotNull() &
    col("user_id").isNotNull()
)

# 2) Enforce rating to be integer in [1..5]
df = df.withColumn("rating_int", col("rating").cast("int"))
df = df.filter(
    col("rating_int").isNotNull() &
    (col("rating_int") >= 1) &
    (col("rating_int") <= 5)
)

# 3) Normalize text; drop empty or ultra-short reviews (<10 chars after trim)
df = df.withColumn("review_text", trim(col("review_text")))
df = df.filter(
    col("review_text").isNotNull() &
    (length(col("review_text")) >= 10)
)

# 4) De-duplicate by review_id
df = df.dropDuplicates(["review_id"])

# 5) Select final shape
reviews_clean = df.select(
    "review_id",
    "book_id",
    "user_id",
    col("rating_int").alias("rating"),
    "review_text"
)

In [0]:
reviews_clean.write.mode("overwrite").parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/"
)

reviews_verified = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/"
)
reviews_verified.printSchema()
reviews_verified.show(5, truncate=False)
print(f"Written cleaned rows: {reviews_verified.count()}")

In [0]:
%sql
USE CATALOG hive_metastore;

In [0]:
books.write.format("delta").mode("overwrite").save("abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/books_delta/")
spark.sql("CREATE TABLE books USING DELTA LOCATION 'abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/books_delta/'")

In [0]:
authors.write.format("delta").mode("overwrite").save("abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/authors_delta/")
spark.sql("CREATE TABLE authors USING DELTA LOCATION 'abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/authors_delta/'")

In [0]:
dbutils.fs.ls("abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/")

In [0]:
from pyspark.sql.functions import col, length, trim, count, when

# c. Read reviews (Parquet) and profile
reviews = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/"
)

reviews.show(5, truncate=False)
reviews.printSchema()

total_rows = reviews.count()
null_review_id = reviews.filter(col("review_id").isNull()).count()
null_book_id = reviews.filter(col("book_id").isNull()).count()
null_user_id = reviews.filter(col("user_id").isNull()).count()
null_rating = reviews.filter(col("rating").isNull()).count()
empty_text = reviews.filter( (col("review_text").isNull()) | (trim(col("review_text")) == "") ).count()

print(f"Total rows: {total_rows}")
print(f"NULL review_id: {null_review_id}, NULL book_id: {null_book_id}, NULL user_id: {null_user_id}, NULL rating: {null_rating}")
print(f"Empty/NULL review_text: {empty_text}")

In [0]:
from pyspark.sql.functions import col, trim, length

# d. Clean reviews by removing problematic rows
df = reviews

df = df.filter(
    col("review_id").isNotNull() &
    col("book_id").isNotNull() &
    col("user_id").isNotNull()
)

df = df.withColumn("rating_int", col("rating").cast("int"))
df = df.filter(
    col("rating_int").isNotNull() &
    (col("rating_int") >= 1) &
    (col("rating_int") <= 5)
)

df = df.withColumn("review_text", trim(col("review_text")))
df = df.filter(
    col("review_text").isNotNull() &
    (length(col("review_text")) >= 10)
)

df = df.dropDuplicates(["review_id"])

reviews_clean = df.select(
    "review_id",
    "book_id",
    "user_id",
    col("rating_int").alias("rating"),
    "review_text"
)

reviews_clean.show(5)

In [0]:
# e. Persist Cleaned Reviews
reviews_clean.write.mode("overwrite").parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/"
)

reviews_verified = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/"
)
reviews_verified.printSchema()
reviews_verified.show(5, truncate=False)
print(f"Written cleaned rows: {reviews_verified.count()}")

In [0]:
# Reload clean Parquet to bypass cache
reviews_clean_fresh = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/"
)

# Optional: Unpersist old df if cached
reviews_clean.unpersist()

# Now write/register Delta from fresh df
reviews_clean_fresh.write.format("delta").mode("overwrite").save("abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews_delta/")
spark.sql("CREATE TABLE IF NOT EXISTS reviews_clean USING DELTA LOCATION 'abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews_delta/'")

In [0]:
%sql
DESCRIBE reviews_clean;
SELECT * FROM reviews_clean LIMIT 5;

In [0]:
%sql
DROP TABLE IF EXISTS reviews_clean;

In [0]:
reviews_clean = spark.read.parquet("abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/")

In [0]:
from pyspark.sql.functions import col, length, trim, count, when

reviews = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/"
)

reviews.show(5, truncate=False)
reviews.printSchema()

total_rows = reviews.count()
null_review_id = reviews.filter(col("review_id").isNull()).count()
null_book_id = reviews.filter(col("book_id").isNull()).count()
null_user_id = reviews.filter(col("user_id").isNull()).count()
null_rating = reviews.filter(col("rating").isNull()).count()
empty_text = reviews.filter( (col("review_text").isNull()) | (trim(col("review_text")) == "") ).count()

print(f"Total rows: {total_rows}")
print(f"NULL review_id: {null_review_id}, NULL book_id: {null_book_id}, NULL user_id: {null_user_id}, NULL rating: {null_rating}")
print(f"Empty/NULL review_text: {empty_text}")

In [0]:
from pyspark.sql.functions import col, trim, length

df = reviews

df = df.filter(
    col("review_id").isNotNull() &
    col("book_id").isNotNull() &
    col("user_id").isNotNull()
)

df = df.withColumn("rating_int", col("rating").cast("int"))
df = df.filter(
    col("rating_int").isNotNull() &
    (col("rating_int") >= 1) &
    (col("rating_int") <= 5)
)

df = df.withColumn("review_text", trim(col("review_text")))
df = df.filter(
    col("review_text").isNotNull() &
    (length(col("review_text")) >= 10)
)

df = df.dropDuplicates(["review_id"])

reviews_clean = df.select(
    "review_id",
    "book_id",
    "user_id",
    col("rating_int").alias("rating"),
    "review_text"
)

reviews_clean.show(5)

In [0]:
reviews_clean.write.mode("overwrite").parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/"
)

reviews_verified = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/"
)
reviews_verified.printSchema()
reviews_verified.show(5, truncate=False)
print(f"Written cleaned rows: {reviews_verified.count()}")

In [0]:
# Reload to fix stale cache
reviews = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews/"
)

# Re-clean (run d code again if needed, but use fresh reviews)
from pyspark.sql.functions import col, trim, length

df = reviews

df = df.filter(
    col("review_id").isNotNull() &
    col("book_id").isNotNull() &
    col("user_id").isNotNull()
)

df = df.withColumn("rating_int", col("rating").cast("int"))
df = df.filter(
    col("rating_int").isNotNull() &
    (col("rating_int") >= 1) &
    (col("rating_int") <= 5)
)

df = df.withColumn("review_text", trim(col("review_text")))
df = df.filter(
    col("review_text").isNotNull() &
    (length(col("review_text")) >= 10)
)

df = df.dropDuplicates(["review_id"])

reviews_clean = df.select(
    "review_id",
    "book_id",
    "user_id",
    col("rating_int").alias("rating"),
    "review_text"
)

# Write Delta from fresh df
reviews_clean.write.format("delta").mode("overwrite").save("abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews_delta/")
spark.sql("CREATE TABLE IF NOT EXISTS reviews_clean USING DELTA LOCATION 'abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/processed/reviews_delta/'")

In [0]:
# Verify table now exists
spark.sql("REFRESH TABLE reviews_clean")

In [0]:
%%sql
DESCRIBE reviews_clean;
SELECT * FROM reviews_clean LIMIT 5;

In [0]:
####HomeWork1
from pyspark.sql.functions import col, lit
from pyspark.sql.types import TimestampType

curated = reviews_clean.join(books, on="book_id", how="inner") \
                       .join(authors, on="author_id", how="inner") \
                       .select(
                           reviews_clean["review_id"],
                           reviews_clean["book_id"],
                           books["title"],
                           authors["author_id"],
                           authors["name"].alias("author_name"),
                           reviews_clean["user_id"],
                           reviews_clean["rating"],
                           reviews_clean["review_text"],
                           books["language_code"].alias("language"),
                           lit(0).alias("n_votes"),
                           lit(None).cast(TimestampType()).alias("date_added")
                       ).dropDuplicates()

curated.printSchema()
curated.show(5, truncate=False)
print(f"Curated rows: {curated.count()}")

In [0]:

###HomeWork1
curated.write.format("delta").mode("overwrite").save("abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/gold/curated_reviews/")

spark.sql("""
    CREATE TABLE IF NOT EXISTS curated_reviews USING DELTA LOCATION 'abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/gold/curated_reviews/'
""")

In [0]:
###HomeWork1
%%sql
SELECT * FROM curated_reviews LIMIT 5